# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

In [8]:
query1 = "SELECT artist, song, length from artists_by_session where session_id = 338 and item_in_session = 4"                    

For this query `session_id` and` item_in_session` are considered composite keys, because the analysis performed seeks to evaluate a particular session item (item_in_session) within a specific session (session_id) and so each item is an execution within this session.

#### Create table artists_by_session

In [9]:
query = "CREATE TABLE IF NOT EXISTS artists_by_session"
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert data

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        query = "INSERT INTO artists_by_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Query test

In [11]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2

In [12]:
query2 = "SELECT artist, song, user from artists_of_user where user_id = 10 and session_id = 182"

For this query `user_id` and` session_id` are considered composite keys, because the analysis performed seeks to evaluate sessions (session_id) for a given user (user_id). With this, it is possible to see the data of the most heard artists and songs within a specific session for a user, or the most heard artists/songs for a user, or even the main artists/songs heard for users in general. Finally, the `item_in_session` field is used as a clustering column, so the data is sorted by this field.

#### Create table artists_of_user

In [13]:
query = "CREATE TABLE IF NOT EXISTS artists_of_user"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, user text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Insert data

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        query = "INSERT INTO artists_of_user (user_id, session_id, item_in_session, artist, song, user)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        user = line[1] + ' ' + line[4]
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], user))

#### Query test

In [15]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3

In [16]:
query3 = "SELECT user from users_song where song='All Hands Against His Own'"

For this query `song` and` user` are considered composite keys because the analysis performed seeks to evaluate the songs for each user. Thus it is possible, quickly, to return which are the songs that a specific user listens to or to search for which are the users who heard a specific song.

#### Create table users_song

In [17]:
query = "CREATE TABLE IF NOT EXISTS users_song"
query = query + "(song text, user text, PRIMARY KEY (song, user))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert table

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        query = "INSERT INTO users_song (song, user)"
        query = query + " VALUES (%s, %s)"
        user = line[1] + ' ' + line[4]
        session.execute(query, (line[9], user))

#### Query test

In [19]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [20]:
query = "drop table if exists artists_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists artists_of_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists users_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()